In [10]:
import DataSetQuery
import datetime


query = DataSetQuery.DataSetQuery('http://localhost:9000')

#Sets the output directory for the getnetcdffile call
query.setEnvironment('test','/data/puma1/scratch/malard/export/')

print(query.getEnvironment())

print(query.getParentDataSets())

print(query.getDataSets('mtngla'))

print(query.getDataSetBoundingBox( 'mtngla','tandemx' ))
print(query.getDataSetBoundingBox( 'mtngla','srtm' ))


minX=0
maxX=100000
minY=0
maxY=100000
minT=datetime.datetime(2010,7,1,0,0)
maxT=datetime.datetime(2010,12,31,0,0)

gcs = query.getGridCells( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT )
#print(gcs)

shards = query.getShards('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT)
#print(shards)

#print(query.getNetCdfFile('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT))

{"name":"test","outputCdfPath":"/data/puma1/scratch/malard/export/"}
{"dataSets":[{"name":"mtngla"},{"name":"tempstore"}]}
{"dataSets":[{"name":"srtm"},{"name":"tandemx"}]}
{"gridCellMinX":-1900000,"gridCellMaxX":1700000,"gridCellMinY":-1300000,"gridCellMaxY":900000,"minTime":1279266673000,"maxTime":1546296307000,"totalPoints":5307063627,"numberOfShards":53566}
{"gridCellMinX":-1900000,"gridCellMaxX":1700000,"gridCellMinY":-1300000,"gridCellMaxY":900000,"minTime":1279266673000,"maxTime":1546296307000,"totalPoints":5307063627,"numberOfShards":53828}


In [11]:
from pandas.io.json import json_normalize
import json


data = json.loads(gcs)
df = json_normalize(data['boxes'])

print(df)

totalPoints = df['totalPoints'].sum()
maxNumPoints = df['totalPoints'].max()

print('TP=[%d] MaxPoints in Cell=[%d]'%(totalPoints,maxNumPoints))   



   gridCellMaxX  gridCellMaxY  gridCellMinX  gridCellMinY        maxTime  \
0        200000        200000        100000        100000  1292715924000   
1             0        200000       -100000        100000  1293061242000   
2        100000             0             0       -100000  1292888583000   
3        200000             0        100000       -100000  1292543264000   
4        100000        100000             0             0  1293061242000   
5             0        100000       -100000             0  1293233898000   
6        200000        100000        100000             0  1292543264000   
7        100000        200000             0        100000  1292888583000   
8             0             0       -100000       -100000  1293061242000   

         minTime  numberOfShards  totalPoints  
0  1279824627000               5       588262  
1  1280515145000               6       950622  
2  1280169839000               5       903633  
3  1279997229000               5       828783  

In [12]:

dataShard = json.loads(shards)


dfShards = json_normalize(dataShard['shards'])

print(dfShards['shardName'])


0    /data/snail1/scratch/tandemx/swath/y2010/m9/ce...
1    /data/snail1/scratch/tandemx/swath/y2010/m11/c...
2    /data/snail1/scratch/tandemx/swath/y2010/m10/c...
3    /data/snail1/scratch/tandemx/swath/y2010/m12/c...
4    /data/snail1/scratch/tandemx/swath/y2010/m7/ce...
Name: shardName, dtype: object


In [13]:
print(query.getDataSetColumns('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT))

http://localhost:9000/point/datasetcolumns/mtngla/tandemx
{"column":[{"name":"lon"},{"name":"lat"},{"name":"elev"},{"name":"heading"},{"name":"demDiff"},{"name":"demDiffMad"},{"name":"demDiffMad2"},{"name":"phaseAmb"},{"name":"meanDiffSpread"},{"name":"wf_number"},{"name":"sampleNb"},{"name":"power"},{"name":"powerdB"},{"name":"phase"},{"name":"phaseS"},{"name":"phaseSSegment"},{"name":"phaseConfidence"},{"name":"coh"},{"name":"x"},{"name":"y"},{"name":"time"},{"name":"swathFileId"}]}


In [16]:
import MalardHelpers

print("Query Malard for a NetCDF for the bounding box")

fileName = query.getNetCdfFile('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT)

print("File created: %s" % (fileName))

df = MalardHelpers.getDataFrameFromNetCDF(fileName)

print(df[0])


Query Malard for a NetCDF for the bounding box
File created: /data/puma1/scratch/malard/export/mtngla_tandemx_0_100000_0_100000_1277938800000_1293753600000.nc
lon                8.509294e+01
lat                3.686482e+01
elev               5.470720e+03
heading            1.856812e+02
demDiff            2.030366e+01
demDiffMad         6.417168e+01
demDiffMad2        4.012519e+01
phaseAmb           0.000000e+00
meanDiffSpread     3.698232e+02
wf_number          3.730000e+02
sampleNb           3.700000e+01
power              2.329794e+03
powerdB           -1.742081e+02
phase              1.888965e+00
phaseS                      NaN
phaseSSegment               NaN
phaseConfidence             NaN
coh                6.180000e-01
x                  8.135808e+03
y                  9.775551e+04
time               1.285560e+09
swathFileId        9.721000e+03
Name: 0, dtype: float64
